In [9]:
from selenium import webdriver
from tempfile import mkdtemp
import sqlalchemy
import requests
import numpy as np
import pandas as pd
import datetime
import json
import urllib
from sqlalchemy import create_engine
from selenium_stealth import stealth
import pymysql


URL_SEARCH = "https://www.tickpick.com/concerts/yeat-tickets/"

options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument("--disable-gpu")
options.add_argument("--window-size=1280x1696")
options.add_argument("--single-process")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--disable-dev-tools")
options.add_argument("--no-zygote")
options.add_argument(f"--user-data-dir={mkdtemp()}")
options.add_argument(f"--data-path={mkdtemp()}")
options.add_argument(f"--disk-cache-dir={mkdtemp()}")
options.add_argument("--remote-debugging-port=9222")

driver = webdriver.Chrome(r"C:\Users\Paul\Documents\chromedriver.exe", options=options)

stealth(driver,
    languages=["en-US", "en"],
    vendor="Google Inc.",
    platform="Win32",
    webgl_vendor="Intel Inc.",
    renderer="Intel Iris OpenGL Engine",
    fix_hairline=True,
    )

driver.get(URL_SEARCH)

xpath_links = "//*[@id=\"events\"]/div[*]/a[1]"

links = driver.find_elements_by_xpath(xpath_links)

venue_names = driver.find_elements_by_xpath("//*[@id=\"events\"]/div[*]/div[2]/span/span")
dates = driver.find_elements_by_xpath("//*[@id=\"events\"]/div[*]/div[1]/span[1]")

links = list(map(lambda elem: elem.get_attribute("href"), links))
venue_names = list(map(lambda elem: elem.text, venue_names))
dates = list(map(lambda elem: elem.text, dates))

def add_page_to_row_list(link, row_list, driver, venue_name, date, time):
    driver.get(link)

    xpath = "//*[@id=\"listingContainer\"]/div[*]"

    elements = []

    while len(elements) == 0:
        elements = driver.find_elements_by_xpath(xpath)

    for elem in elements:
        dic = {}
        dic["Date"] = time
        dic["VenueID"] = venue_name
        dic["DateEvent"] = date
        dic["Quantity"] = elem.find_element_by_tag_name("select").text.split()[0]
        dic["Price"] = elem.find_element_by_tag_name("b").text[1:]
        row_list.append(dic)

row_list = []

date = datetime.datetime.now()
time = date.strftime("%m.%d.%y_%H.%M")

for i in range(len(links)):
    print(links[i])
    add_page_to_row_list(links[i], row_list, driver, venue_names[i], dates[i], time)

df = pd.DataFrame(row_list)
df.head()

df["Price"].head(100)

server = 'tickpick-scraper-database.mysql.database.azure.com'
database = 'tickpick_database'
username = 'pauldoucet'
driver = '{ODBC Driver 17 for SQL Server}'
ca_path = "DigiCertGlobalRootCA.crt.pem"

f = open("password.json")
data = json.load(f)
password = data["password"]

sqlUrl = URL(
    drivername="mysql+pymysql",
    username=username,
    password=password,
    host=server,
    port=3306,
    database=database,
    query={"ssl_ca": ca_path},
)
engine = create_engine(sqlUrl)

df.to_sql("test_table", con=engine, if_exists="append", index=False)

<ipython-input-9-3fbc9cf3f382>:30: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(r"C:\Users\Paul\Documents\chromedriver.exe", options=options)
<ipython-input-9-3fbc9cf3f382>:45: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead
  links = driver.find_elements_by_xpath(xpath_links)
<ipython-input-9-3fbc9cf3f382>:47: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead
  venue_names = driver.find_elements_by_xpath("//*[@id=\"events\"]/div[*]/div[2]/span/span")
<ipython-input-9-3fbc9cf3f382>:48: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead
  dates = driver.find_elements_by_xpath("//*[@id=\"events\"]/div[*]/div[1]/span[1]")


https://www.tickpick.com/buy-yeat-tickets-concord-music-hall-4-7-22-6pm/5080801/


<ipython-input-9-3fbc9cf3f382>:62: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead
  elements = driver.find_elements_by_xpath(xpath)
C:\Users\Paul\miniconda3\lib\site-packages\selenium\webdriver\remote\webelement.py:341: UserWarning: find_element_by_tag_name is deprecated. Please use find_element(by=By.TAG_NAME, value=name) instead
  warnings.warn("find_element_by_tag_name is deprecated. Please use find_element(by=By.TAG_NAME, value=name) instead")


https://www.tickpick.com/buy-yeat-tickets-el-club-4-8-22-9pm/5076275/
https://www.tickpick.com/buy-yeat-tickets-newport-music-hall-4-9-22-7pm/5076846/
https://www.tickpick.com/buy-yeat-tickets-the-fillmore-silver-spring-4-12-22-8pm/5076024/
https://www.tickpick.com/buy-yeat-tickets-brooklyn-bowl-philadelphia-4-13-22-8pm/5076025/
https://www.tickpick.com/buy-yeat-tickets-the-summit-music-hall-4-15-22-8pm/5076844/
https://www.tickpick.com/buy-yeat-tickets-brighton-music-hall-ma-4-17-22-7pm/5076023/
https://www.tickpick.com/buy-yeat-tickets-irving-plaza-4-18-22-7pm/5076021/
https://www.tickpick.com/buy-yeat-tickets-warsaw-4-19-22-7pm/5076022/
https://www.tickpick.com/buy-yeat-tickets-heaven-stage-at-masquerade-ga-4-21-22-6pm/5076845/
https://www.tickpick.com/buy-yeat-tickets-the-underground-charlotte-4-24-22-8pm/5076276/
https://www.tickpick.com/buy-yeat-tickets-bronze-peacock-at-the-house-of-blues-houston-4-27-22-8pm/5076240/
https://www.tickpick.com/buy-yeat-tickets-the-echo-lounge-and-

AttributeError: type object 'URL' has no attribute 'create'